In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
from datetime import date, timedelta
import pandas as pd
from bs4 import BeautifulSoup
import re
import string
import numpy as np
import os

In [2]:
# Establecemos las rutas de lectura y escritura de ficheros
write_path = '../data/news.csv'
read_path = '../data/urls.csv'

In [3]:
df_news = pd.read_csv(write_path)
df_news['scraping_date'] = pd.to_datetime(df_news['scraping_date'], format="%Y/%m/%d")
dates = df_news['scraping_date']
keep = dates > (np.datetime64(date.today() - timedelta(days=5)))
df_news = df_news[keep]
scrapped_url = df_news['url']
scrapped_url

df_urls = pd.read_csv(read_path)
urls = list(df_urls['url'])

In [4]:
# Creamos la clase Spider
# Extrayendo el titular y el texto de la noticia, menos en los periódicos de suscripción

class news_spider(scrapy.Spider):
    name = 'blogspider'
    start_urls = urls
    
    # Inicalizamos el fichero de escritura con las cabeceras
    def __init__(self):
        if os.stat(write_path).st_size == 0:
            print(f"scraping_date,url,headline,text,lang", file=filep)

    def parse(self, response):

        list_replace = ['\n', '"', ',']
        extract_text = ""
        lang = 'es'
        
        today = False
        if response.url in scrapped_url:
            today = True

        print(f"\"{today}\",\"{response.url}\",\"headline\",\"clean_text\",\"eslang\"", file=filep)           


In [8]:
url = "hola"
scrapped_url.append("hola")
url in scrapped_url

False

In [7]:
scrapped_url

0       https://www.20minutos.es/noticia/4393682/0/ult...
1       https://www.20minutos.es/noticia/4395865/0/mad...
2       https://www.20minutos.es/noticia/4395732/0/ayu...
3       https://www.20minutos.es/noticia/4395580/0/com...
4       https://www.20minutos.es/noticia/4395560/0/rec...
                              ...                        
2227    https://www.news.com.au/sport/cricket/alyssa-h...
2228    https://www.news.com.au/sport/motorsport/formu...
2229    https://www.news.com.au/sport/sports-life/elli...
2230    https://www.news.com.au/sport/football/worlds-...
2231    https://www.news.com.au/sport/boxing-champion-...
Name: url, Length: 2232, dtype: object

In [5]:
# Lanzamos el proceso de Crawler
process = CrawlerProcess({
    'USER_AGENT': 'Google SEO Bot'
})


filep = open(write_path, 'a')
process.crawl(news_spider)
process.start()
filep.close()

25657/article/2020-09-17/liege-laube-des-grands-changements>
2020-09-27 12:30:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://demainlaterre.lesoir.be/322019/article/2020-08-31/les-caf-ou-le-microcredit-pour-sentraider> (referer: None)
2020-09-27 12:30:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://plus.lesoir.be/327143/article/2020-09-23/comment-paul-rusesabagina-heros-du-film-hotel-rwanda-est-arrive-au-rwanda-au> (referer: None)
2020-09-27 12:30:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://plus.lesoir.be/325657/article/2020-09-17/liege-laube-des-grands-changements> (referer: None)
2020-09-27 12:30:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://plus.lesoir.be/320677/article/2020-08-24/des-remorques-velo-partager> (referer: None)
2020-09-27 12:30:31 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://plus.lesoir.be/323918/article/2020-09-09/maroc-mohammed-vi-le-roi-des-pauvres-toujours-plus-loin-de-son-peuple> fro

In [7]:
df_news = pd.read_csv(write_path)
df_news.head(10)

,scraping_date,url,headline,text,lang
0,2020-09-10,https://www.20minutos.es/noticia/4375204/0/dav...,David Escors: Estamos yendo más rápido de lo q...,A raíz de la suspensión de los ensayos clínic...,es
1,2020-09-10,https://www.20minutos.es/noticia/4375385/0/un-...,Un vídeo muestra lo lejos que puede llegar el ...,Que el coronavirus se expande de forma rápid...,es
2,2020-09-10,https://www.20minutos.es/noticia/4373963/0/ast...,Asturias el 'oasis' en una España barrida por ...,Asturias se ha convertido también en esta segu...,es
3,2020-09-10,https://www.20minutos.es/noticia/4375219/0/la-...,La Policía dice que Interior pagó 53.000 euros...,El informe de la Policía Nacional que ha rec...,es
4,2020-09-10,https://www.20minutos.es/noticia/4375386/0/dir...,Casado: La votación es un triunfo del municipa...,El Pleno del Congreso ha rechazado este jueves...,es
5,2020-09-10,https://www.20minutos.es/noticia/4362007/0/los...,Las servilletas de bar la otra víctima que el ...,Las medidas antiCovid del Gobierno central ...,es
6,2020-09-10,https://www.20minutos.es/noticia/4375035/0/res...,El camino de la Covid desemboca otra vez en la...,Fueron los más castigados por la primera ola d...,es
7,2020-09-10,https://www.20minutos.es/noticia/4375123/0/edu...,Educación deja sin plaza a docentes por cumpli...,La Confederación General del Trabajo (CGT) a...,es
8,2020-09-10,https://www.20minutos.es/noticia/4375388/0/oli...,Olivia Molina: Si hubiera una valla yo estaría...,En la serie La valla de Antena 3 (que se estre...,es
9,2020-09-10,https://www.20minutos.es/noticia/4375384/0/cas...,Así es el casoplón que Cristina Pedroche y Dab...,Cristina Pedroche y Dabiz Muñoz han dejado s...,es
